In [ ]:
spfs = hf.system.spfs;
C = hf.C;
C_tilde = C';
l = hf.system.l
h = hf.system.h
u = hf.system.u;

In [ ]:
function transform_spfs(spfs, C)
    spfs2 = zero.(spfs)
    l = length(spfs)
    
    for i in 1:l
        psi1 = spfs2[i]
        for j in 1:l
            psi1 .+= C[j, i] .* spfs[j]
        end
    end
    return spfs2
end

In [ ]:
@time spfs2 = transform_spfs(spfs, C);

In [ ]:
function transform_onebody(h, C)
    h2 = zero(h)
    l = size(C)[1]
    
    @inbounds for a in 1:l
        for b in 1:l
            for i in 1:l
                for j in 1:l
                    h2[a, b] += C[i, a] * C[j, b] * h[i, j]
                end
            end
        end
    end
    return h2
end

In [ ]:
function transform_onebody2(h, C)
    return C' * h * C;
end

In [ ]:
@time h2 = transform_onebody(h, C);

In [ ]:
@time h3 = transform_onebody2(h, C);

In [ ]:
h2 ≈ h3

## Twobody

In [2]:
l = 6
u = rand(l,l,l,l);
C = rand(l,l);

In [10]:
@time transform_twobody_slow(u, C);
@time transform_twobody(u, C);
transform_twobody_slow(u, C) ≈ transform_twobody(u, C)

  0.002960 seconds (3 allocations: 10.406 KiB)
  0.000036 seconds (4 allocations: 20.594 KiB)


true

In [4]:
function transform_twobody_slow(u, C)
    u2 = zero(u)
    l = length(C[:,1])
    
    for a in 1:l, b in 1:l, c in 1:l, d in 1:l
        for i in 1:l, j in 1:l, k in 1:l, m in 1:l
            @inbounds u2[a, b, c, d] += C[i, a] * C[j, b] * C[k, c] * C[m, d] * u[i, j, k, m]
        end
    end
    return u2
end

transform_twobody_slow (generic function with 1 method)

In [ ]:
@time u_slow = transform_twobody_slow(u, C);

In [62]:
function transform_twobody(u, C)
    l = size(C)[1]
    
    u2 = zero(u)
    #u3 = zero(u)
    @inbounds Threads.@threads for i in 1:l
        Ci = C[:, i]
        for b in 1:l, c in 1:l, a in 1:l
            s = 0.0
            for j in 1:l
                s += Ci[j] * u[a, b, c, j]
            end
            u2[a, b, c, i] = s
        end
    end
    #=
    @inbounds Threads.@threads for i in 1:l
        Ci = C[:, i]
        for b in 1:l, d in 1:l, a in 1:l
            s = 0.0
            for j in 1:l
                s += Ci[j] * u2[a, b, j, d]
            end
            u3[a, b, i, d] = s
        end
    end
    
    @inbounds Threads.@threads for i in 1:l
        Ci = C[:, i]
        for c in 1:l, d in 1:l, a in 1:l
            s = 0.0
            for j in 1:l
                s += Ci[j] * u3[a, j, c, d]
            end
            u2[a, i, c, d] = s
        end
    end
    
    @inbounds Threads.@threads for i in 1:l
        Ci = C[:, i]
        for c in 1:l, d in 1:l, b in 1:l
            s = 0.0
            for j in 1:l
                s += Ci[j] * u2[j, b, c, d]
            end
            u3[i, b, c, d] = s
        end
    end
    =#
    return u2
end

transform_twobody (generic function with 1 method)

In [64]:
l = 48
u = rand(l,l,l,l);
C = rand(l,l);

@time transform_twobody_nice(u, C);
@time transform_twobody(u, C);
transform_twobody_nice(u, C) ≈ transform_twobody(u, C)
@profview transform_twobody(u, C)

  0.023754 seconds (3 allocations: 40.500 MiB, 47.10% gc time)
  0.142857 seconds (231 allocations: 40.537 MiB)


ProfileCanvas.ProfileData(Dict{String, ProfileCanvas.ProfileFrame}("4" => ProfileCanvas.ProfileFrame("root", "", "", 0, 0, missing, 0x00, missing, ProfileCanvas.ProfileFrame[]), "1" => ProfileCanvas.ProfileFrame("root", "", "", 0, 70, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#1", "threadingconstructs.jl", ".\\threadingconstructs.jl", 30, 55, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#997#threadsfor_fun", "threadingconstructs.jl", ".\\threadingconstructs.jl", 51, 55, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#997#threadsfor_fun#28", "threadingconstructs.jl", ".\\threadingconstructs.jl", 84, 55, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("macro expansion", "In[62]", ".\\In[62]", 11, 43, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("+", "float.jl", ".\\float.jl", 383, 31, missing, 0x00, missing, ProfileCanvas.ProfileFrame[]), ProfileCanvas.ProfileFrame("getindex", "array.jl", ".\\array.jl", 925, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[]), ProfileCanvas.ProfileFrame("getindex", "array.jl", ".\\array.jl", 0, 4, missing, 0x00, missing, ProfileCanvas.ProfileFrame[]), ProfileCanvas.ProfileFrame("*", "float.jl", ".\\float.jl", 385, 2, missing, 0x00, missing, ProfileCanvas.ProfileFrame[])]), ProfileCanvas.ProfileFrame("macro expansion", "In[62]", ".\\In[62]", 12, 12, missing, 0x00, missing, ProfileCanvas.ProfileFrame[])])])]), ProfileCanvas.ProfileFrame("#666", "task.jl", ".\\task.jl", 134, 9, missing, 0x10, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("macro expansion", "asyncevent.jl", ".\\asyncevent.jl", 279, 9, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("_trywait", "asyncevent.jl", ".\\asyncevent.jl", 138, 9, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("wait", "condition.jl", ".\\condition.jl", 124, 9, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("wait", "task.jl", ".\\task.jl", 930, 9, missing, 0x10, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("poptask", "task.jl", ".\\task.jl", 921, 9, missing, 0x10, missing, ProfileCanvas.ProfileFrame[])])])])])]), ProfileCanvas.ProfileFrame("#15", "task.jl", ".\\task.jl", 484, 6, missing, 0x10, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("eventloop", "eventloop.jl", "C:\\Users\\KarlH\\.julia\\packages\\IJulia\\AQu2H\\src\\eventloop.jl", 8, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("invokelatest", "essentials.jl", ".\\essentials.jl", 726, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#invokelatest#2", "essentials.jl", ".\\essentials.jl", 729, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("execute_request", "execute_request.jl", "C:\\Users\\KarlH\\.julia\\packages\\IJulia\\AQu2H\\src\\execute_request.jl", 67, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("softscope_include_string", "SoftGlobalScope.jl", "C:\\Users\\KarlH\\.julia\\packages\\SoftGlobalScope\\u4UzH\\src\\SoftGlobalScope.jl", 65, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("include_string", "loading.jl", ".\\loading.jl", 1428, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("eval", "boot.jl", ".\\boot.jl", 368, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("transform_twobody", "In[62]", ".\\In[62]", 6, 3, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("macro expansion", "threadingconstructs.jl", ".\\threadingconstructs.jl", 89, 3, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("threading_run", "threadingconstructs.jl", ".\\threadingconstructs.jl", 38, 3, missing, 0x00, missing, Profi

In [ ]:
39.139 ms (119 allocations: 39.09 MiB)

In [7]:
using ProfileCanvas

In [31]:
using BenchmarkTools: @btime

In [ ]:
4.940 ms (300 allocations: 2.47 MiB)

In [ ]:
0.319575 seconds (516 allocations: 39.111 MiB, 3.72% gc time)

In [37]:
l = 20
u = rand(l,l,l,l);
C = rand(l,l);

@time u2 = transform_twobody(u, C);
@time u3 = transform_twobody_slow(u, C);
@time u3 = transform_twobody_nice(u, C);
u2 ≈ u3

  0.013534 seconds (6 allocations: 2.442 MiB)
 68.212012 seconds (4 allocations: 1.221 MiB)
  0.003254 seconds (545 allocations: 7.372 MiB)


true

In [ ]:
u2 ≈ u_py

In [ ]:
using PyCall

In [3]:
using TensorOperations

In [4]:
function transform_twobody_nice(u, C)
    @tensor begin
        u2[a, b, c, i] := u[a, b, c, d] * C[d, i]
        #u3[a, b, i, d] := u2[a, b, c, d] * C[c, i]
        #u2[a, i, c, d] = u3[a, b, c, d] * C[b, i]
        #u3[i, b, c, d] = u2[a, b, c, d] * C[a, i]
    end
    return u2
end

transform_twobody_nice (generic function with 1 method)

In [ ]:
println(fasit ≈ u3)

In [ ]:
py"""
import numpy as np

def please(u, C, C_tilde):
    _u = np.tensordot(u, C, axes=(3, 0))
    _u = np.tensordot(_u, C, axes=(2, 0)).transpose(0, 1, 3, 2)
    _u = np.tensordot(_u, C_tilde, axes=(1, 1)).transpose(0, 3, 1, 2)
    _u = np.tensordot(C_tilde, _u, axes=(1, 0))
    return _u
"""
@time fasit = py"please"(u, C, C');

In [ ]:
findmax(fasit), findmin(fasit)